In [ ]:
#LLM Evalutation 의 중요성

서비스를 안정적으로 운영하기 위해서
사용자가 정확한 정보를 받을 수 있는지


"할루시네이션" ->환각
LLM 이 잘못된 답변을 생성하는 경우
"안전"-> "잘못된 정보"->사고

"방지하기 위한 최소한의 도구"

#Dataset -> 도메인 전문가가 작성한 "정답지"
특정 질문이 들어오면 -> 이런 답변을 해야한다


In [1]:
%pip install langsmith openai python-dotenv langchain langchain-openai langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [4]:
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'

database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)
retriever = database.as_retriever()

c:\Users\soobi\.conda\envs\selenium_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_openai import ChatOpenAI
from langsmith import traceable

llm = ChatOpenAI(model="gpt-4o", temperature=1)

# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = "".join(doc.page_content for doc in docs)
    instructions = f"""당신은 한국의 소득세 전문가 입니다.아래 소득세법을 참고해서 사용자의 질문에 답변해주세요.

Documents:
{docs_string}"""
    # langchain ChatModel will be automatically traced
    ai_msg = llm.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )
    return {"answer": ai_msg.content, "documents": docs}

In [9]:
from langsmith import Client

client = Client()

# Define the examples for the dataset
examples = [
  {
    "inputs": {
      "question": "소득세법상 '거주자'의 정의는 무엇이며, 비거주자와 어떻게 구분하나요?"
    },
    "outputs": {
      "answer": "거주자는 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말하며, 비거주자는 거주자가 아닌 개인을 의미합니다.",
      "context": "제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다. 2. “비거주자”란 거주자가 아닌 개인을 말한다."
    }
  },
  {
    "inputs": {
      "question": "거주자의 종합소득에는 어떠한 종류의 소득이 합산되나요?"
    },
    "outputs": {
      "answer": "종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득, 기타소득을 모두 합산한 소득을 의미합니다.",
      "context": "제4조(소득의 구분) ① 거주자의 소득은 다음 각 호와 같이 구분한다. 1. 종합소득: 이 법에 따라 과세되는 모든 소득에서 제2호(퇴직소득) 및 제3호(양도소득)에 따른 소득을 제외한 소득으로서 다음 각 목의 소득을 합산한 것 (가. 이자소득, 나. 배당소득, 다. 사업소득, 라. 근로소득, 마. 연금소득, 바. 기타소득)"
    }
  },
  {
    "inputs": {
      "question": "일반적인 경우 소득세의 과세기간은 어떻게 되나요?"
    },
    "outputs": {
      "answer": "소득세의 과세기간은 매년 1월 1일부터 12월 31일까지 1년으로 합니다.",
      "context": "제5조(과세기간) ① 소득세의 과세기간은 1월 1일부터 12월 31일까지 1년으로 한다."
    }
  },
  {
    "inputs": {
      "question": "근로소득자가 받는 식사대 중 비과세되는 한도는 얼마인가요?"
    },
    "outputs": {
      "answer": "근로자가 사내급식 등을 제공받지 않는 경우, 월 20만원 이하의 식사대는 비과세 소득에 해당합니다.",
      "context": "제12조(비과세소득) 3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득... 러. 근로자가 사내급식이나 이와 유사한 방법으로 제공받는 식사 기타 음식물 또는 근로자(식사 기타 음식물을 제공받지 아니하는 자에 한정한다)가 받는 월 20만원 이하의 식사대"
    }
  },
  {
    "inputs": {
      "question": "인적공제 중 기본공제를 받기 위한 부양가족의 소득 요건은 무엇인가요?"
    },
    "outputs": {
      "answer": "거주자와 생계를 같이 하는 부양가족으로서 해당 과세기간의 소득금액 합계액이 100만원 이하(총급여액 500만원 이하의 근로소득만 있는 경우 포함)여야 합니다.",
      "context": "제50조(기본공제) ① ... 3. 거주자(그 배우자를 포함한다)와 생계를 같이 하는 다음 각 목의 어느 하나에 해당하는 부양가족으로서 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하의 근로소득만 있는 부양가족을 포함한다)"
    }
  },
  {
    "inputs": {
      "question": "종합소득 과세표준 확정신고는 언제까지 해야 하나요?"
    },
    "outputs": {
      "answer": "해당 과세기간의 종합소득금액이 있는 거주자는 그 다음 연도 5월 1일부터 5월 31일까지 납세지 관할 세무서장에게 신고해야 합니다.",
      "context": "제70조(종합소득과세표준 확정신고) ① 해당 과세기간의 종합소득금액이 있는 거주자는 그 종합소득 과세표준을 그 과세기간의 다음 연도 5월 1일부터 5월 31일까지 대통령령으로 정하는 바에 따라 납세지 관할 세무서장에게 신고하여야 한다."
    }
  },
  {
    "inputs": {
      "question": "자녀세액공제의 대상 나이와 자녀 수에 따른 공제 금액은 어떻게 되나요?"
    },
    "outputs": {
      "answer": "8세 이상의 자녀나 손자녀가 대상이며, 1명일 때 연 25만원, 2명일 때 연 55만원, 3명 이상인 경우 연 55만원에 2명을 초과하는 1명당 연 40만원을 합산하여 공제합니다.",
      "context": "제59조의2(자녀세액공제) ① 종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다."
    }
  },
  {
    "inputs": {
      "question": "소득세법에서 '사업자'란 누구를 의미하나요?"
    },
    "outputs": {
      "answer": "사업소득이 있는 거주자를 말합니다.",
      "context": "제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. 5. “사업자”란 사업소득이 있는 거주자를 말한다."
    }
  },
  {
    "inputs": {
      "question": "거주자가 사망한 경우의 소득세 과세기간은 어떻게 계산하나요?"
    },
    "outputs": {
      "answer": "거주자가 사망한 경우에는 1월 1일부터 사망한 날까지를 과세기간으로 합니다.",
      "context": "제5조(과세기간) ② 거주자가 사망한 경우의 과세기간은 1월 1일부터 사망한 날까지로 한다."
    }
  },
  {
    "inputs": {
      "question": "거주자가 출국하여 비거주자가 되는 경우 과세기간은 어떻게 되나요?"
    },
    "outputs": {
      "answer": "거주자가 주소 또는 거소를 국외로 이전하여 비거주자가 되는 경우, 과세기간은 1월 1일부터 출국한 날까지로 합니다.",
      "context": "제5조(과세기간) ③ 거주자가 주소 또는 거소를 국외로 이전(이하 “출국”이라 한다)하여 비거주자가 되는 경우의 과세기간은 1월 1일부터 출국한 날까지로 한다."
    }
  },
  {
    "inputs": {
      "question": "거주자의 소득세 납세지는 원칙적으로 어디인가요?"
    },
    "outputs": {
      "answer": "거주자의 소득세 납세지는 그 주소지로 하며, 주소지가 없는 경우에는 그 거소지로 합니다.",
      "context": "제6조(납세지) ① 거주자의 소득세 납세지는 그 주소지로 한다. 다만, 주소지가 없는 경우에는 그 거소지로 한다."
    }
  },
  {
    "inputs": {
      "question": "국내사업장이 없는 비거주자의 소득세 납세지는 어디로 결정되나요?"
    },
    "outputs": {
      "answer": "국내사업장이 없는 경우에는 국내원천소득이 발생하는 장소를 납세지로 합니다.",
      "context": "제6조(납세지) ② 비거주자의 소득세 납세지는... 국내사업장이 없는 경우에는 국내원천소득이 발생하는 장소로 한다."
    }
  },
  {
    "inputs": {
      "question": "소득세법의 목적은 무엇인가요?"
    },
    "outputs": {
      "answer": "개인의 소득에 대해 소득 성격과 부담능력에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입을 원활하게 조달하는 것을 목적으로 합니다.",
      "context": "제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다."
    }
  },
  {
    "inputs": {
      "question": "공동사업에서 발생하는 소득금액에 대한 납세의무는 누가 지나요?"
    },
    "outputs": {
      "answer": "공동사업에 관한 소득금액을 계산하는 경우에는 해당 공동사업자별로 납세의무를 집니다.",
      "context": "제2조의2(납세의무의 범위) ① 제43조에 따라 공동사업에 관한 소득금액을 계산하는 경우에는 해당 공동사업자별로 납세의무를 진다."
    }
  },
  {
    "inputs": {
      "question": "피상속인의 소득금액에 대해 과세하는 경우 누구에게 납세의무가 있나요?"
    },
    "outputs": {
      "answer": "피상속인의 소득금액에 대해서 과세하는 경우에는 그 상속인이 납세의무를 집니다.",
      "context": "제2조의2(납세의무의 범위) ② 제44조에 따라 피상속인의 소득금액에 대해서 과세하는 경우에는 그 상속인이 납세의무를 진다."
    }
  },
  {
    "inputs": {
      "question": "신탁재산에 귀속되는 소득의 납세의무자는 누구인가요?"
    },
    "outputs": {
      "answer": "신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 수익자(수익자가 사망한 경우 상속인)에게 귀속되는 것으로 봅니다.",
      "context": "제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) ① 신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 수익자(수익자가 사망하는 경우에는 그 상속인)에게 귀속되는 것으로 본다."
    }
  },
  {
    "inputs": {
      "question": "외국인 거주자 중 국내 거주 기간이 짧은 경우 국외 발생 소득에 대한 과세 범위는 어떻게 되나요?"
    },
    "outputs": {
      "answer": "국내에 주소나 거소를 둔 기간의 합계가 5년 이하인 외국인 거주자는 국외 발생 소득 중 국내에서 지급되거나 국내로 송금된 소득에 대해서만 과세합니다.",
      "context": "제3조(과세소득의 범위) ① ...해당 과세기간 종료일 10년 전부터 국내에 주소나 거소를 둔 기간의 합계가 5년 이하인 외국인 거주자에게는 과세대상 소득 중 국외에서 발생한 소득의 경우 국내에서 지급되거나 국내로 송금된 소득에 대해서만 과세한다."
    }
  },
  {
    "inputs": {
      "question": "법인으로 보는 단체 외의 법인 아닌 단체의 납세의무는 어떻게 적용되나요?"
    },
    "outputs": {
      "answer": "국내에 주사무소를 둔 경우 1거주자로, 그 외에는 1비거주자로 보아 법을 적용하며 이익 분배 비율이 확인되는 경우 구성원별로 납세의무를 집니다.",
      "context": "제2조(납세의무) ③ ...법인으로 보는 단체 외의 법인 아닌 단체는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 경우에는 1거주자로, 그 밖의 경우에는 1비거주자로 보아 이 법을 적용한다."
    }
  },
  {
    "inputs": {
      "question": "비거주자는 어떤 소득에 대해서 소득세를 납부할 의무가 있나요?"
    },
    "outputs": {
      "answer": "비거주자는 소득세법 제119조에 따른 국내원천소득이 있는 경우에 한하여 소득세를 납부할 의무를 집니다.",
      "context": "제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다. 2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인"
    }
  },
  {
    "inputs": {
      "question": "공동으로 소유한 자산에 대한 양도소득금액 계산 시 납세의무자는 누구인가요?"
    },
    "outputs": {
      "answer": "공동으로 소유한 자산에 대한 양도소득금액을 계산하는 경우에는 해당 자산을 공동으로 소유하는 각 거주자가 납세의무를 집니다.",
      "context": "제2조의2(납세의무의 범위) ⑤ 공동으로 소유한 자산에 대한 양도소득금액을 계산하는 경우에는 해당 자산을 공동으로 소유하는 각 거주자가 납세의무를 진다."
    }
  }
]
  

# Create the dataset and examples in LangSmith
daTaset_name = "Korean tax Q&A"
dataset = client.create_dataset(dataset_name=daTaset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['6950184f-9a4c-48cd-97e2-81bad510ed3b',
  '34afb5d7-fa7b-471b-a3f1-3116ecf265b7',
  'bd1ae345-719e-4b08-a726-a47ac6c0220c',
  'ba45a428-b3ef-4f90-8365-6da5fa255d98',
  '16ef33a1-fef0-4aa8-8e49-9f7ded15cb7e',
  '2fb59a89-1b3c-427c-a9d1-183276eb6e63',
  '7a0f9df2-00f3-4633-93f7-d8d6ada67213',
  'ad10b52f-6bb4-44e1-a6ff-aeb6886a91c0',
  '958fd021-8c72-4347-84b1-0f3a43512ccd',
  'b4768e6b-92f4-4d1a-9f02-e55344ad1d4c',
  '79079e67-f23c-49bc-8f8f-a82dba9ef3a3',
  'c094edb9-0797-4d67-a45c-ae47e514e3d1',
  'ed296d4a-a0f8-428b-8986-dea19e621d6e',
  'a635b176-9964-4a2f-86d8-5db3e03c8046',
  'c0f2c0e6-1255-441a-85fe-7c2c6bdd8f8b',
  '87158164-e35a-4da3-b3d0-7b057a373dd6',
  'f4bb311b-47be-4148-a795-006a0c80c0fd',
  '8dbad306-12c1-4a19-bc55-955be9f084b4',
  '65ca8e73-c67c-44ce-8863-f09e43ef38c1',
  '906e40c7-4315-4d5e-9f62-536ee44534c1'],
 'count': 20}

In [10]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. (2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=True
)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""
    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

In [13]:

# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[
        bool, ..., "Provide the score on if the answer hallucinates from the documents"
    ]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. You will be given FACTS and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. (2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    GroundedGrade, method="json_schema", strict=True
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

In [14]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

dataset_name = "Korean tax Q&A"
experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness],
    experiment_prefix="tax-doc-relevance",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)

# Explore results locally as a dataframe if you have pandas installed
# experiment_results.to_pandas()

View the evaluation results for experiment: 'tax-doc-relevance-02388c7a' at:
https://smith.langchain.com/o/a3341128-8e57-43dc-96e1-772e0292c7ee/datasets/3c1f132d-17cd-4119-9715-8ea22ec965cf/compare?selectedSessions=817315fe-3977-46c2-a1ae-4faf74b685fb




20it [04:50, 14.53s/it]
